In [1]:
from molecules import PPS, PolyEthylene
from system import System
from simulation import Simulation
import foyer
import warnings
warnings.filterwarnings("ignore")

# Using the recipes in molecules.py:

molecules.py contains classes for different monomers.  
These classes can be thought of as recipes or templates
to quickly build up polymer chains of any length

In [17]:
# Use the PPS template to create a 5mer PPS molecule
pps_chain = PPS(length=5)
pps_chain.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
# We can do the same with a Poly-ethylene chain
pe_chain = PolyEthylene(length=8)
pe_chain.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Building up a system of polymers:
Pass in an instance of a polymer template, and a list of number of molecules and a list of molecule lengths

In [18]:
sys = System(molecule=PolyEthylene, n_mols=[10], chain_lengths=[10], density=1.3)

In [19]:
# Call one of the configuration building functions; right now, we only have pack()
sys.pack()

In [21]:
# Apply a foyer forcefield
opls = foyer.Forcefield(name="oplsaa")
sys.apply_forcefield(forcefield=opls)

In [22]:
# We now have 2 different versions of our system. An mBuild Compound and a Parmed Structure
print(sys.system)
print()
print(sys.typed_system)

<Compound 620 particles, 610 bonds, System box: Box: Lx=76.696126, Ly=76.696126, Lz=76.696126, xy=0.000000, xz=0.000000, yz=0.000000, , id: 6452434976>

<Structure 620 atoms; 1 residues; 610 bonds; PBC (orthogonal); parametrized>


# Starting up a simulation of our polymer system
- Use the Simulation class to quickly get a simulation going.  
- Pass in a `System.typed_system` which has all the forcefield information
- Most of the simulation properties of interest are easily accessble, and changeable via methods in the Simulation class
- The integrator and integrator method is determined by which run funciton you call
```
run_NVT
run_NPT
run_NVE
run_langevin
```

In [10]:
sim = Simulation(system=sys.typed_system)

Processing LJ and QQ
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing RB torsions


In [11]:
print(sim.nlist)
print()
print(sim.forcefield)
print()
print(sim.dt)


[<hoomd.md.pair.pair.LJ object at 0x180ef5640>, <hoomd.md.pair.pair.Ewald object at 0x180ef5760>, <hoomd.md.long_range.pppm.Coulomb object at 0x180f1f1c0>, <hoomd.md.special_pair.LJ object at 0x180ef5550>, <hoomd.md.special_pair.Coulomb object at 0x180f1fa60>, <hoomd.md.bond.Harmonic object at 0x180f1fe20>, <hoomd.md.angle.Harmonic object at 0x180f244f0>, <hoomd.md.dihedral.OPLS object at 0x1044edc70>]

0.0003


In [12]:
# The integrator and integrator method are not set up until a run function is called for the first time:
sim.run_NVT(n_steps=0, kT=1, tau_kt=0.1)

notice(2): charge.pppm: RMS error: 0.0982533


In [13]:
print(sim.integrator)

In [14]:
print(sim.method)

In [15]:
# Change simulation parameters like dt
print(sim.integrator.dt)
sim.dt = 0.0001
print(sim.integrator.dt)

0.0003
0.0001


In [16]:
# Calling a run function again will update the method if needed
print(sim.method)
sim.run_NPT(n_steps=0, pressure=0.001, kT=1.0, tau_kt=0.01, tau_pressure=0.1)
print(sim.method)

# Combining the run functions to get an equilibrated system
- First, call the `run_shrink` function to get the box down to the desired density
- Next, run in the NPT ensemble to get an equilibrated volume
- Finally, run at NVT to get an equilibrated morphology

In [23]:
# Build up a polyethylene system of 10 10mers, apply an OPLS forcefield
sys = System(molecule=PolyEthylene, n_mols=[10], chain_lengths=[10], density=1.3)
sys.pack()
opls = foyer.Forcefield(name="oplsaa")
sys.apply_forcefield(forcefield=opls)

In [25]:
# We can use the target_box attribute to help with the shrink run
# Remember we'll have to convert from nm to angstroms
# Remember to account for the reference distance if using auto scaling
sys.target_box

array([1.53392282, 1.53392282, 1.53392282])

In [26]:
sim = Simulation(system=sys.typed_system, auto_scale=True)

Processing LJ and QQ
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing RB torsions


In [28]:
# This is in angstroms:
sim.ref_distance

3.5

In [29]:
sim.run_shrink(
    kT=5.0,
    final_box_lengths=sys.target_box * 10 / sim.ref_distance,
    n_steps=1e4,
    tau_kt = 0.01,
    period=100,
)

notice(2): charge.pppm: RMS error: 0.0982533


**ERROR**: Particle with unique tag 588 is no longer in the simulation box.

Cartesian coordinates: 
x: -0.901592 y: -40.148 z: 2.05055
Fractional coordinates: 
f.x: 0.361934 f.y: -5.64811 f.z: 0.814013
Local box lo: (-3.26506, -3.26506, -3.26506)
          hi: (3.26506, 3.26506, 3.26506)


RuntimeError: Error computing cell list